# Kayak Flight Data

Data we need:
Main cities
- nonstop, one-way domestic - how many? - Valerie
- Kayak gives 10: NYC, Vegas, LA, Chicago, SF, Orlando, Denver, Washington DC, Seattle, Boston

Departure time/date -- 12/22 up to 1/1
- Filter non-stop

Figure out url for loop (url format)

Kayak’s advice - - AMY

Airlines available for the flight- Jon 

Ticket price-- Nicole


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import pandas
import re

In [3]:
url = 'https://www.kayak.com'
r = requests.get(url)
layer1Cookies = r.cookies

url2 = url + '/s/horizon/flights/results/FlightSearchPoll'

origin_code = 'WAS'
origin_location = 'WASHINGTON'
destination_code = 'AMD'
destination_location = 'Ahmedabad'
depart_date = '2018-12-20'
return_date = '2019-01-27'

referer = 'https://www.kayak.com/flights/'+origin_code+'-'+destination_code+'/'+depart_date+'/'+return_date
headers = {
    'Host': 'www.kayak.com',
    'User-Agent': 'Chrome/63 (Macintosh; Intel Mac OS X 10.11; rv:57.0) Gecko/20100101 Firefox/57.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': referer,
    'Content-Type': 'application/x-www-form-urlencoded',
    'X-CSRF': 'kAqI1NgGh$DJnEUpiSDOWpdQXzlgAwG8EVOCd$gXO08-hpumC4oNpaOjz15GO_q9a5FdZPonpC2kF4CBYjEPh14',
    'X-RequestId': 'flights#frontdoor#Ag$s9g',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Length': '1094'
}

url3 = 'flights/'+origin_code+'-'+destination_code+'/'+depart_date+'/'+return_date

params1 = {
    'searchId':'',
    'poll':'true',
    'pollNumber':'0',
    'applyFilters':'true',
    'filterState':'',
    'useViewStateFilterState':'false',
    'pageNumber':'1',
    'append':'false',
    'pollingId':'593601',  #interesting. explore further
    'requestReason':'POLL',
    'isSecondPhase':'false',
    'textAndPageLocations':'bottom,right',
    'displayAdPageLocations':'none',
    'existingAds':'false',
    'activeLeg':'-1',
    'view':'list',
    'renderPlusMinusThreeFlex':'false',
    'renderAirlineStopsMatrix':'false',
    'renderFlexHeader':'true',
    'tab':'flights',
    'pageOrigin':'F..FD..M0',
    'src':'',
    'searchingAgain':'',
    'c2s':'',
    'po':'',
    'personality':'',
    'provider':'',
    'isMulticity':'false',
    'flex_category':'exact',
    'depart_date':depart_date,
    'return_date':return_date,
    'oneway':'false',
    'origincode':origin_code,  #change accordingly
    'origin':origin_code, #change accordingly
    'origin_location':origin_location, #change accordingly
    'origin_code':'', #change accordingly
    'nearby_origin':'false',
    'destination':destination_code, #change accordingly
    'destination_location':destination_location, #change accordingly
    'destination_code':'', #change accordingly
    'nearby_destination':'false',
    'countrySearch':'false',
    'depart_date_canon':depart_date, #change accordingly
    'return_date_canon':return_date, #change accordingly
    'travelers':'1',
    'adults':'1',
    'seniors':'0',
    'youth':'0',
    'child':'0',
    'seatInfant':'0',
    'lapInfant':'0',
    'cabin':'e',
    'cabinDisplayType':'Economy',
    'vertical':'flights',
    'url':url3,
    'id':'',
    'navigateToResults':'false',
    'ajaxts':'',
    'scriptsMetadata':'',
    'stylesMetadata':'',
}


result = requests.post(url2, headers = headers, data = params1, cookies = layer1Cookies)
text = result.json()

text = BeautifulSoup(text['content'], 'html.parser')

In [14]:
def get_prices_from_page(text):
    doc = html.fromstring(str(text))
    flights = doc.xpath('//div[@class="Base-Results-HorizonResult Flights-Results-FlightResultItem phoenix-rising sleek rp-contrast "]')
    #print(flights)

    result = []
    for f in flights:
        price_info = f.get('aria-label')
        time_info = f.xpath('.//div//div//div//div[@class="col-info result-column"]')[0]
        time = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="depart-time base-time"]')[0].text
        am_pm = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="time-meridiem meridiem"]')[0].text
        
        regex_price = re.compile(r'(\$)(\d*)',re.VERBOSE)
        matches_price = regex_price.finditer(price_info)
        for match in matches_price:
            clean_price = match.group(2)
            entry = {
                'price' : clean_price,
                'time' : time + am_pm
            }
            result.append(entry)
        
    return result

In [15]:
get_prices_from_page(text)

[{'price': '1845', 'time': '10:55 pm'},
 {'price': '2062', 'time': '6:35 pm'},
 {'price': '1845', 'time': '10:55 pm'},
 {'price': '1870', 'time': '10:55 pm'},
 {'price': '1870', 'time': '10:55 pm'},
 {'price': '1870', 'time': '10:55 pm'},
 {'price': '1870', 'time': '10:55 pm'},
 {'price': '1883', 'time': '10:55 pm'},
 {'price': '1893', 'time': '10:55 pm'},
 {'price': '1893', 'time': '10:55 pm'},
 {'price': '1968', 'time': '6:35 pm'},
 {'price': '2019', 'time': '6:35 pm'},
 {'price': '2055', 'time': '6:35 pm'},
 {'price': '2082', 'time': '6:35 pm'},
 {'price': '2097', 'time': '10:55 pm'}]